In [15]:
import mysql.connector
import pandas as pd

In [16]:
conn = mysql.connector.connect(
    host = 'database-1.cbwuowec22t2.ap-northeast-2.rds.amazonaws.com',
    port = 3306,
    user = 'team2',
    password = "1234",
    database = 'eda_project' 
)

cur = conn.cursor(buffered=True)

In [17]:
query = """
    SELECT g.gu_name, d.dong_name, COUNT(h.hospital_name)
    FROM gu g, dong d, hospital h
    WHERE (g.gu_id = d.gu_id AND d.dong_id = h.dong_id)
    AND h.hospital_name LIKE '%소아%'
    GROUP BY d.dong_name
"""


df = pd.read_sql(query, conn)

df

/tmp/ipykernel_6729/738711711.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,gu_name,dong_name,COUNT(h.hospital_name)
0,강남구,개포동,9
1,강남구,논현동,2
2,강남구,대치동,6
3,강남구,도곡동,2
4,강남구,삼성동,2
...,...,...,...
166,중랑구,면목동,4
167,중랑구,묵동,1
168,중랑구,상봉동,3
169,중랑구,신내동,4


In [18]:
df['rank'] = df['COUNT(h.hospital_name)'].rank(ascending=True, method='min')

# 순위를 기준으로 정렬
df_desc = df.sort_values(by='rank')

# 확인
print(df_desc[['gu_name', 'dong_name','COUNT(h.hospital_name)', 'rank']])

    gu_name dong_name  COUNT(h.hospital_name)   rank
11      강남구       자곡동                       1    1.0
9       강남구       율현동                       1    1.0
23      강북구       수유동                       1    1.0
16      강동구       둔촌동                       1    1.0
61      동작구      노량진동                       1    1.0
..      ...       ...                     ...    ...
0       강남구       개포동                       9  167.0
49      노원구       상계동                      10  168.0
128     양천구        목동                      10  168.0
130     양천구       신정동                      10  168.0
32      관악구       봉천동                      13  171.0

[171 rows x 4 columns]


In [19]:
hos_list = [(row['gu_name'], row['dong_name'],row['rank']) for _, row in df_desc.iterrows()]

In [20]:
hos_list

[('강남구', '자곡동', 1.0),
 ('강남구', '율현동', 1.0),
 ('강북구', '수유동', 1.0),
 ('강동구', '둔촌동', 1.0),
 ('동작구', '노량진동', 1.0),
 ('구로구', '오류동', 1.0),
 ('광진구', '군자동', 1.0),
 ('광진구', '광장동', 1.0),
 ('성북구', '석관동', 1.0),
 ('성북구', '삼선동4가', 1.0),
 ('송파구', '풍납동', 1.0),
 ('성북구', '종암동', 1.0),
 ('송파구', '석촌동', 1.0),
 ('송파구', '오금동', 1.0),
 ('성북구', '동소문동1가', 1.0),
 ('성북구', '돈암동', 1.0),
 ('성동구', '금호동3가', 1.0),
 ('성동구', '도선동', 1.0),
 ('성동구', '상왕십리동', 1.0),
 ('서대문구', '북아현동', 1.0),
 ('서초구', '신원동', 1.0),
 ('성동구', '금호동1가', 1.0),
 ('서대문구', '홍은동', 1.0),
 ('서초구', '양재동', 1.0),
 ('서대문구', '연희동', 1.0),
 ('마포구', '신공덕동', 1.0),
 ('마포구', '성산동', 1.0),
 ('마포구', '용강동', 1.0),
 ('마포구', '창전동', 1.0),
 ('마포구', '중동', 1.0),
 ('마포구', '신수동', 1.0),
 ('마포구', '대흥동', 1.0),
 ('성북구', '보문동5가', 1.0),
 ('성북구', '동소문동2가', 1.0),
 ('성동구', '성수동2가', 1.0),
 ('용산구', '원효로1가', 1.0),
 ('용산구', '후암동', 1.0),
 ('은평구', '녹번동', 1.0),
 ('은평구', '증산동', 1.0),
 ('은평구', '역촌동', 1.0),
 ('종로구', '누하동', 1.0),
 ('영등포구', '문래동6가', 1.0),
 ('용산구', '도원동', 1.0),
 ('종로구', '숭인동', 1.0),
 ('중

In [21]:
def update_rank_dong_hos(gu, dong, rank):
    # 구 ID 가져오기
    cur.execute('SELECT gu_id FROM gu WHERE gu_name = %s', (gu,))
    gu_id = cur.fetchone()[0]

    # 동 ID 가져오기
    cur.execute('SELECT dong_id FROM dong WHERE dong_name = %s', (dong,))
    dong_id = cur.fetchone()[0]


    # gu_id에 해당하는 row가 존재하면 업데이트
    cur.execute(
        'UPDATE rank_for_dong SET rank_hospital = %s WHERE gu_id = %s AND dong_id = %s', 
        (rank, gu_id, dong_id))
    
    conn.commit()


In [22]:
for gu, dong, rank in hos_list:
    update_rank_dong_hos(gu, dong, rank)